In [1]:
# Import useful packages
import os
import time
import pandas as pd
import glob
import numpy as np

# For loading/querying postgres
from sqlalchemy import create_engine
import psycopg2
from psycopg2 import connect

In [2]:
# Create SQL engine to load data to postgres for central usage
from sqlalchemy import create_engine
dbname = '697_temp'
user = 'postgres'
password = 'poRter!5067'
host = 'databasesec.cvhiyxfodl3e.us-east-2.rds.amazonaws.com'

engine = create_engine(f'postgresql://{user}:{password}@{host}/{dbname}')

### Loading Open Positions to Postgres

In [3]:
# Collect all CSVs in Open_Positions_Data folder
txtfiles = []
for file in glob.glob("Open_Positions_Data/*.csv"):
    txtfiles.append(file)
    
txtfiles

['Open_Positions_Data\\MF_MLP.csv']

In [4]:
# create temp_df to test loading; pick model not already in table
temp_df = pd.read_csv(txtfiles[0],index_col=0)
temp_df.head()

,key,Date,Ticker,Quantity,Basis,Purchase Price,Current Value,Last,% Gain,model
0,Positions_2019-10-01,2019-10-01,WMT,1577,181331.779861,114.985276,183245.217818,116.198616,0.0,MF_MLP
1,Positions_2019-10-01,2019-10-01,MA,577,155262.373861,269.085570,172734.634810,299.366785,0.0,MF_MLP
2,Positions_2019-10-01,2019-10-01,CVX,1201,128996.841743,107.407862,134148.637533,111.697450,0.0,MF_MLP
3,Positions_2019-10-01,2019-10-01,BA,301,112606.490025,374.107940,98960.939837,328.773887,0.0,MF_MLP
4,Positions_2019-10-01,2019-10-01,COST,253,69469.070068,274.581305,71281.639717,281.745612,0.0,MF_MLP


In [5]:
temp_df.columns

Index(['key', 'Date', 'Ticker', 'Quantity', 'Basis', 'Purchase Price',
       'Current Value', 'Last', '% Gain', 'model'],
      dtype='object')

In [74]:
# Initial dataframe needs to create a new table
#temp_df.to_sql('open_positions', engine)

In [6]:
# Append data to table
temp_df.to_sql('open_positions', engine, if_exists='append')

In [7]:
# Test with a load
def import_open_positions():
    conn = connect(dbname = '697_temp', user = 'postgres', host = 'databasesec.cvhiyxfodl3e.us-east-2.rds.amazonaws.com', password = 'poRter!5067')
    cur = conn.cursor()
    query = "SELECT * FROM open_positions"
    data = pd.read_sql_query(query,conn)
    data = data.sort_values(['key', 'model'])
    return data
open_pos_df = import_open_positions().drop('index',axis=1)

open_pos_df[open_pos_df['model']=='MF_MLP']


,key,Date,Ticker,Quantity,Basis,Purchase Price,Current Value,Last,% Gain,model
18347,Positions_2019-10-01,2019-10-01,WMT,1577,181331.779861,114.985276,183245.217818,116.198616,0.00,MF_MLP
18348,Positions_2019-10-01,2019-10-01,MA,577,155262.373861,269.085570,172734.634810,299.366785,0.00,MF_MLP
18349,Positions_2019-10-01,2019-10-01,CVX,1201,128996.841743,107.407862,134148.637533,111.697450,0.00,MF_MLP
18350,Positions_2019-10-01,2019-10-01,BA,301,112606.490025,374.107940,98960.939837,328.773887,0.00,MF_MLP
18351,Positions_2019-10-01,2019-10-01,COST,253,69469.070068,274.581305,71281.639717,281.745612,0.00,MF_MLP
...,...,...,...,...,...,...,...,...,...,...
23380,Positions_2021-06-30,2021-04-01,MS,1562,121273.898075,77.640140,142102.950596,90.975000,17.18,MF_MLP
23381,Positions_2021-06-30,2021-04-01,C,1660,119877.321160,72.215254,116473.898354,70.164999,-2.84,MF_MLP
23382,Positions_2021-06-30,2021-04-01,BA,438,111560.792473,254.705006,104350.215134,238.242500,-6.46,MF_MLP
23383,Positions_2021-06-30,2021-04-01,GS,283,92071.567490,325.341228,106226.878014,375.359993,15.37,MF_MLP


In [8]:
df = open_pos_df[open_pos_df['model']=='MF_MLP']

In [9]:
df

,key,Date,Ticker,Quantity,Basis,Purchase Price,Current Value,Last,% Gain,model
18347,Positions_2019-10-01,2019-10-01,WMT,1577,181331.779861,114.985276,183245.217818,116.198616,0.00,MF_MLP
18348,Positions_2019-10-01,2019-10-01,MA,577,155262.373861,269.085570,172734.634810,299.366785,0.00,MF_MLP
18349,Positions_2019-10-01,2019-10-01,CVX,1201,128996.841743,107.407862,134148.637533,111.697450,0.00,MF_MLP
18350,Positions_2019-10-01,2019-10-01,BA,301,112606.490025,374.107940,98960.939837,328.773887,0.00,MF_MLP
18351,Positions_2019-10-01,2019-10-01,COST,253,69469.070068,274.581305,71281.639717,281.745612,0.00,MF_MLP
...,...,...,...,...,...,...,...,...,...,...
23380,Positions_2021-06-30,2021-04-01,MS,1562,121273.898075,77.640140,142102.950596,90.975000,17.18,MF_MLP
23381,Positions_2021-06-30,2021-04-01,C,1660,119877.321160,72.215254,116473.898354,70.164999,-2.84,MF_MLP
23382,Positions_2021-06-30,2021-04-01,BA,438,111560.792473,254.705006,104350.215134,238.242500,-6.46,MF_MLP
23383,Positions_2021-06-30,2021-04-01,GS,283,92071.567490,325.341228,106226.878014,375.359993,15.37,MF_MLP


### Loading track records to postgres

In [10]:
# Collect all CSVs in Open_Positions_Data folder
txtfiles = []
for file in glob.glob("track_record/*.csv"):
    txtfiles.append(file)
    
txtfiles

['track_record\\MF_MLP.csv']

In [11]:
# create temp_df to test loading
temp_df = pd.read_csv(txtfiles[0],index_col=0)
temp_df.head()

,Date,Value,Val_ex_cash,model
0,2019-10-01,1000000.000000,997715.670596,MF_MLP
1,2019-10-02,978527.170952,976242.841548,MF_MLP
2,2019-10-03,975356.274485,973071.945080,MF_MLP
3,2019-10-04,991904.663615,989620.334211,MF_MLP
4,2019-10-07,995212.910401,992928.580997,MF_MLP


In [81]:
# Initial dataframe needs to create a new table
#temp_df.to_sql('track_record', engine)

In [12]:
# Append data to table
temp_df.to_sql('track_record', engine, if_exists='append')

In [13]:
# Test with a load
def import_track_record():
    conn = connect(dbname = '697_temp', user = 'postgres', host = 'databasesec.cvhiyxfodl3e.us-east-2.rds.amazonaws.com', password = 'poRter!5067')
    cur = conn.cursor()
    query = "SELECT * FROM track_record"
    data = pd.read_sql_query(query,conn)
    data = data.sort_values(['Date', 'model'])
    return data
track_df = import_track_record().drop('index',axis=1)

track_df[track_df['model']=='MF_MLP']


,Date,Value,Val_ex_cash,model
782,2019-10-01,1.000000e+06,9.977157e+05,MF_MLP
783,2019-10-02,9.785272e+05,9.762428e+05,MF_MLP
784,2019-10-03,9.753563e+05,9.730719e+05,MF_MLP
785,2019-10-04,9.919047e+05,9.896203e+05,MF_MLP
786,2019-10-07,9.952129e+05,9.929286e+05,MF_MLP
...,...,...,...,...
1218,2021-06-24,1.489047e+06,1.488434e+06,MF_MLP
1219,2021-06-25,1.506016e+06,1.505403e+06,MF_MLP
1220,2021-06-28,1.499579e+06,1.498966e+06,MF_MLP
1221,2021-06-29,1.503010e+06,1.502397e+06,MF_MLP


### Loading cash records to postgres

In [14]:
# Collect all CSVs in Open_Positions_Data folder
txtfiles = []
for file in glob.glob("Cash_Positions_Data/*.csv"):
    txtfiles.append(file)
    
txtfiles

['Cash_Positions_Data\\MF_MLP.csv']

In [15]:
# create temp_df to test loading
temp_df = pd.read_csv(txtfiles[0],index_col=0)
temp_df

,key,cash,model
0,cash_2019-10-01,2284.329404,MF_MLP
1,cash_2019-10-02,2284.329404,MF_MLP
2,cash_2019-10-03,2284.329404,MF_MLP
3,cash_2019-10-04,2284.329404,MF_MLP
4,cash_2019-10-07,2284.329404,MF_MLP
...,...,...,...
436,cash_2021-06-24,612.778087,MF_MLP
437,cash_2021-06-25,612.778087,MF_MLP
438,cash_2021-06-28,612.778087,MF_MLP
439,cash_2021-06-29,612.778087,MF_MLP


In [16]:
# Initial dataframe needs to create a new table
temp_df.to_sql('cash_record', engine, if_exists='append')

In [17]:
# Test with a load
def import_cash_record():
    conn = connect(dbname = '697_temp', user = 'postgres', host = 'databasesec.cvhiyxfodl3e.us-east-2.rds.amazonaws.com', password = 'poRter!5067')
    cur = conn.cursor()
    query = "SELECT * FROM cash_record"
    data = pd.read_sql_query(query,conn)
    data = data.sort_values(['model','key'])
    return data
cash_df = import_cash_record().drop('index',axis=1)

cash_df.tail()


,key,cash,model
946,cash_2021-06-14,3.438173,RF Reg_target_7_rebal_7_2018-01-01
947,cash_2021-06-15,3.438173,RF Reg_target_7_rebal_7_2018-01-01
948,cash_2021-06-16,3.438173,RF Reg_target_7_rebal_7_2018-01-01
949,cash_2021-06-17,3.438173,RF Reg_target_7_rebal_7_2018-01-01
950,cash_2021-06-18,3.438173,RF Reg_target_7_rebal_7_2018-01-01


In [18]:
cash_df[cash_df['model']=='MF_MLP']

,key,cash,model
1377,cash_2019-10-01,2284.329404,MF_MLP
1378,cash_2019-10-02,2284.329404,MF_MLP
1379,cash_2019-10-03,2284.329404,MF_MLP
1380,cash_2019-10-04,2284.329404,MF_MLP
1381,cash_2019-10-07,2284.329404,MF_MLP
...,...,...,...
1807,cash_2021-06-24,612.778087,MF_MLP
1808,cash_2021-06-25,612.778087,MF_MLP
1809,cash_2021-06-28,612.778087,MF_MLP
1810,cash_2021-06-29,612.778087,MF_MLP


### Loading Value Positions to postgres

In [19]:
# Collect all CSVs in Open_Positions_Data folder
txtfiles = []
for file in glob.glob("Val_Positions_Data/*.csv"):
    txtfiles.append(file)
    
txtfiles

['Val_Positions_Data\\MF_MLP.csv']

In [23]:
# create temp_df to test loading
temp_df = pd.read_csv(txtfiles[0],index_col=0)
temp_df = temp_df.rename(columns = {'Value':'value'})
temp_df.tail()

,key,value,model
442,val_2021-06-24,1.489047e+06,MF_MLP
443,val_2021-06-25,1.506016e+06,MF_MLP
444,val_2021-06-28,1.499579e+06,MF_MLP
445,val_2021-06-29,1.503010e+06,MF_MLP
446,val_2021-06-30,1.499137e+06,MF_MLP


In [24]:
# Initial dataframe needs to create a new table
temp_df.to_sql('val_positions', engine,if_exists='append')

In [25]:
# Test with a load
def import_val_positions():
    conn = connect(dbname = '697_temp', user = 'postgres', host = 'databasesec.cvhiyxfodl3e.us-east-2.rds.amazonaws.com', password = 'poRter!5067')
    cur = conn.cursor()
    query = "SELECT * FROM val_positions"
    data = pd.read_sql_query(query,conn)
    data = data.sort_values(['model','key'])
    return data
val_df = import_val_positions().drop('index',axis=1)

val_df.tail()

,key,value,model
1570,val_2021-06-14,152965.410119,RF Reg_target_7_rebal_7_2018-01-01
1571,val_2021-06-15,152885.820590,RF Reg_target_7_rebal_7_2018-01-01
1572,val_2021-06-16,151615.342741,RF Reg_target_7_rebal_7_2018-01-01
1573,val_2021-06-17,149310.843319,RF Reg_target_7_rebal_7_2018-01-01
1574,val_2021-06-18,145732.861447,RF Reg_target_7_rebal_7_2018-01-01


In [26]:
val_df[val_df['model'] == 'MF_MLP']

,key,value,model
323,val_2019-10-01,1.000000e+06,MF_MLP
324,val_2019-10-02,9.785272e+05,MF_MLP
405,val_2019-10-03,9.753563e+05,MF_MLP
406,val_2019-10-04,9.919047e+05,MF_MLP
423,val_2019-10-07,9.952129e+05,MF_MLP
...,...,...,...
1405,val_2021-06-24,1.489047e+06,MF_MLP
1521,val_2021-06-25,1.506016e+06,MF_MLP
1603,val_2021-06-28,1.499579e+06,MF_MLP
1685,val_2021-06-29,1.503010e+06,MF_MLP
